In [1]:
%load_ext nbextension
from enum import Enum
import pandas as pd

import s3fs

from gswp.patterns import frozen_singleton
from gswp.geos import HIMAWARI, GEOS

fs = s3fs.S3FileSystem(anon=True)

In [51]:
%load_ext nbextension
from enum import Enum, EnumMeta
from typing import NamedTuple, ParamSpec, Protocol, TypeVar, Union, Mapping, Generic


class StrEnum(str, Enum):
    ...


P = ParamSpec("P")
T1 = TypeVar("T1", bound=StrEnum)
T2 = TypeVar("T2", bound=StrEnum)
K = TypeVar("K", bound=str)


class EnumType(Union[T1, T2], type[StrEnum]):
    ...


The nbextension extension is already loaded. To reload it, use:
  %reload_ext nbextension


In [64]:
def extend(base: type[T1]):
    base_mapping = {member.name: member.value for member in base}

    def wrapps(cls: type[T2]) -> Union[T1, T2]:
        class_mapping = {member.name: member.value for member in cls}
        return StrEnum(cls.__name__, base_mapping | class_mapping)

    return wrapps


class GeosMeta(EnumMeta):
    def __instancecheck__(cls: "GeosType", instance: "GeosType") -> bool:

        return (
            super().__instancecheck__(instance)
            or isinstance(instance, GEOS.EAST)
            or isinstance(instance, GEOS.WEST)
        )


class GeosType(StrEnum, metaclass=GeosMeta):
    ABI_L1B_RADC = "ABI-L1b-RadC"
    ABI_L1B_RADF = "ABI-L1b-RadF"


class GEOS:
    @extend(GeosType)
    class WEST(StrEnum):
        ABI_L2_ACHAC = "ABI-L2-ACHAC"

    @extend(GeosType)
    class EAST(StrEnum):
        ABI_L2_AODC = "ABI-L2-AODC"


assert isinstance(GEOS.EAST.ABI_L1B_RADC, GeosType)


assert isinstance(GEOS.WEST.ABI_L1B_RADC, GeosType)

assert isinstance(GEOS.EAST.ABI_L1B_RADC, GEOS.EAST)
assert isinstance(GEOS.WEST.ABI_L1B_RADC, GEOS.WEST)

assert not isinstance(GEOS.WEST, GEOS.EAST)

assert GEOS.WEST.ABI_L2_ACHAC not in GEOS.EAST and GEOS.WEST.ABI_L2_ACHAC in GEOS.WEST

In [1]:
%load_ext nbextension
from abc import ABCMeta
from typing import Generic, Literal, NewType, Any
from collections import namedtuple
from gswp.geos import HIMAWARI, GEOS, GeosType

from enum import Enum, EnumMeta
from typing import ParamSpec, Protocol


assert isinstance(GEOS.EAST.ABI_L1B_RADM, GEOS)
assert isinstance(GEOS.WEST.ABI_L1B_RADM, GEOS)
assert isinstance(GEOS.EAST.ABI_L2_ACHTM, GEOS.EAST)
assert isinstance(GEOS.WEST.ABI_L2_ACHTM, GEOS.WEST)
assert not isinstance(GEOS.WEST, GEOS.EAST)
assert not isinstance(GEOS.EAST.ABI_L1B_RADM, HIMAWARI)

assert not isinstance(GEOS.WEST.ABI_L2_ACHTM, GEOS.EAST)
assert GEOS.EAST == GEOS.EAST and GEOS.EAST != GEOS.WEST

In [21]:
import xarray as xr
import pandas as pd
import io
import s3fs
from contextlib import contextmanager

REQUEST = GEOS.WEST.ABI_L1B_RADF
fs = s3fs.S3FileSystem(anon=True)


def get_urls(prod: HIMAWARI | GeosType, date: pd.Timestamp) -> list[str]:

    if isinstance(prod, HIMAWARI):
        baseurl = "s3://noaa-himawari8"
        globpattern = f"{date:%m}/{date:%d}/*/*L2P*.nc"
    elif isinstance(prod, GeosType):
        baseurl = f"s3://noaa-goes{16 if isinstance(prod, GEOS.EAST) else 17}"
        globpattern = f"{date.dayofyear:03}/*/*.nc"
    else:
        raise Exception

    return fs.glob(f"{baseurl}/{prod}/{date:%Y}/{globpattern}")


urls = get_urls(REQUEST, pd.Timestamp.fromisoformat("2022-01-01"))


@contextmanager
def open_mfdataset(urls: str):
    def generate():
        for url in urls:
            with fs.open(url, "rb") as f:
                yield io.BytesIO(f.read())

    memory = tuple(generate())
    try:
        yield memory
    finally:
        for buffer in memory:
            buffer.close()


def main():
    with open_mfdataset(urls[:2]) as f:
        ds = xr.open_mfdataset(
            f, combine="nested", concat_dim="time", engine="h5netcdf"
        )
        if isinstance(REQUEST, HIMAWARI):
            rename = {"ni": "lon", "nj": "lat"}
        else:
            rename = {"x": "lon", "y": "lat"}
            ds = ds.rename({"x": "lon", "y": "lat"})
        df = ds.rename(rename).reset_coords().to_dataframe()


urls

['noaa-goes17/ABI-L1b-RadF/2022/001/00/OR_ABI-L1b-RadF-M6C01_G17_s20220010000320_e20220010009386_c20220010009424.nc',
 'noaa-goes17/ABI-L1b-RadF/2022/001/00/OR_ABI-L1b-RadF-M6C01_G17_s20220010010320_e20220010019386_c20220010019431.nc',
 'noaa-goes17/ABI-L1b-RadF/2022/001/00/OR_ABI-L1b-RadF-M6C01_G17_s20220010020320_e20220010029387_c20220010029427.nc',
 'noaa-goes17/ABI-L1b-RadF/2022/001/00/OR_ABI-L1b-RadF-M6C01_G17_s20220010030320_e20220010039387_c20220010039429.nc',
 'noaa-goes17/ABI-L1b-RadF/2022/001/00/OR_ABI-L1b-RadF-M6C01_G17_s20220010040320_e20220010049387_c20220010049430.nc',
 'noaa-goes17/ABI-L1b-RadF/2022/001/00/OR_ABI-L1b-RadF-M6C01_G17_s20220010050320_e20220010059387_c20220010059430.nc',
 'noaa-goes17/ABI-L1b-RadF/2022/001/00/OR_ABI-L1b-RadF-M6C02_G17_s20220010000320_e20220010009386_c20220010009415.nc',
 'noaa-goes17/ABI-L1b-RadF/2022/001/00/OR_ABI-L1b-RadF-M6C02_G17_s20220010010320_e20220010019386_c20220010019415.nc',
 'noaa-goes17/ABI-L1b-RadF/2022/001/00/OR_ABI-L1b-RadF-M